# Prophet vs Seer: Performance at scale (up to 100k rows)

This notebook benchmarks fitting and predicting performance for Prophet and Seer on synthetic time series of increasing length up to 100,000 rows. It mirrors the original `prophet_seer_comparison.ipynb` but focuses on speed and scaling behavior.

Notes:
- The notebook will detect whether `prophet` and `seer` are installed and skip any missing library.
- Seer prefers Polars DataFrames; Prophet uses Pandas.
- The synthetic data includes a linear trend + yearly seasonality + noise.
- For very large sizes, results may take several minutes depending on your machine and configuration.

In [1]:
# Cell 2: Imports and helpers
import time
import math
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

print('Libraries imported')

Libraries imported


In [ ]:
# Cell 3: Check availability of Prophet and Seer
HAS_PROPHET = False
HAS_SEER = False
try:
    from prophet import Prophet  # type: ignore
    HAS_PROPHET = True
    print('✓ Prophet available')
except Exception as e:
    print('⚠ Prophet not available:', e)

try:
    from farseer import Farseer  # type: ignore
    HAS_SEER = True
    print('✓ Seer available')
except Exception as e:
    print('⚠ Seer not available:', e)

# We'll still create the notebook even if one library is missing; cells will skip runs accordingly.

Importing plotly failed. Interactive plots will not work.


✓ Prophet available
⚠ Seer not available: No module named 'seer'


In [3]:
# Cell 4: Synthetic data generator
def make_seasonal_series(n_rows, freq='D', start='2000-01-01', seed=0, amplitude=10.0, trend_slope=0.001, noise_scale=1.0):
    np.random.seed(seed)
    # create equally spaced dates at daily frequency by default; for monthly benchmarks use freq='MS' explicitly
    dates = pd.date_range(start=start, periods=n_rows, freq=freq)
    t = np.arange(n_rows).astype(float)
    # yearly seasonality approximation depending on frequency: use 365.25 for daily, 12 for monthly
    if freq.upper().startswith('D'):
        period = 365.25
    elif freq.upper().startswith('MS') or freq.upper().startswith('M'):
        period = 12.0
    else:
        # fallback to daily-like period scaling
        period = 365.25
    seasonal = amplitude * np.sin(2 * np.pi * t / period)
    trend = trend_slope * t * amplitude
    noise = np.random.normal(scale=noise_scale, size=n_rows)
    y = 100.0 + trend + seasonal + noise
    df = pd.DataFrame({'ds': dates, 'y': y})
    return df

# Quick smoke test
print(make_seasonal_series(10).head())

          ds           y
0 2000-01-01  101.764052
1 2000-01-02  100.582173
2 2000-01-03  101.342719
3 2000-01-04  102.786737
4 2000-01-05  102.595112


In [4]:
# Cell 5: Benchmark loop configuration
sizes = [1000, 5000, 10000, 25000, 50000, 100000]  # up to 100k rows
# Use monthly data for compatibility with the original notebook? Use daily by default for large counts to keep natural spacing
freq = 'D'  # daily makes it easy to scale to 100k rows; change to 'MS' if monthly behavior is desired (note: 100k months is unrealistic)
results = []
max_train_fraction = 0.9  # keep a small test set

print('Benchmark sizes:', sizes)

Benchmark sizes: [1000, 5000, 10000, 25000, 50000, 100000]


In [ ]:
# Cell 6: Run the benchmarks (this cell may take time)
for n in sizes:
    print('' + '='*80)
    print(f'Running benchmark for n={n}')
    df = make_seasonal_series(n_rows=n, freq=freq, seed=42, amplitude=20.0, trend_slope=0.0005, noise_scale=2.0)
    # train/test split
    train_n = max(2, int(n * max_train_fraction))
    df_train = df.iloc[:train_n].reset_index(drop=True)
    df_test = df.iloc[train_n:].reset_index(drop=True)

    # Prepare polars version for Seer
    df_train_pl = pl.from_pandas(df_train)
    df_test_pl = pl.from_pandas(df_test)

    row = {'n': n}

    # Prophet: fit & predict
    if HAS_PROPHET:
        try:
            prophet_model = Prophet(seasonality_mode='additive', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            t0 = time.time()
            prophet_model.fit(df_train)
            fit_t = time.time() - t0
            t0 = time.time()
            # Prophet's predict expects a dataframe with 'ds' column; predict on test period only
            prophet_forecast = prophet_model.predict(df_test[['ds']]) if len(df_test) > 0 else prophet_model.predict(df_train[['ds']])
            predict_t = time.time() - t0
            row.update({'prophet_fit_s': fit_t, 'prophet_predict_s': predict_t})
            print(f'  Prophet: fit={fit_t:.3f}s predict={predict_t:.3f}s')
        except Exception as e:
            print('  Prophet run failed:', e)
            row.update({'prophet_fit_s': None, 'prophet_predict_s': None, 'prophet_error': str(e)})
    else:
        row.update({'prophet_fit_s': None, 'prophet_predict_s': None, 'prophet_error': 'not_installed'})

    # Seer: fit & predict
    if HAS_SEER:
        try:
            seer_model = Farseer(seasonality_mode='additive', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            t0 = time.time()
            seer_model.fit(df_train_pl)
            fit_t = time.time() - t0
            t0 = time.time()
            seer_forecast_pl = seer_model.predict(df_test_pl) if len(df_test_pl) > 0 else seer_model.predict(df_train_pl)
            predict_t = time.time() - t0
            row.update({'seer_fit_s': fit_t, 'seer_predict_s': predict_t})
            print(f'  Seer:   fit={fit_t:.3f}s predict={predict_t:.3f}s')
        except Exception as e:
            print('  Seer run failed:', e)
            row.update({'seer_fit_s': None, 'seer_predict_s': None, 'seer_error': str(e)})
    else:
        row.update({'seer_fit_s': None, 'seer_predict_s': None, 'seer_error': 'not_installed'})

    results.append(row)

print('Benchmark complete')

Running benchmark for n=1000


01:05:38 - cmdstanpy - INFO - Chain [1] start processing
01:05:38 - cmdstanpy - INFO - Chain [1] done processing


  Prophet: fit=0.105s predict=0.030s
Running benchmark for n=5000


01:05:39 - cmdstanpy - INFO - Chain [1] start processing
01:05:39 - cmdstanpy - INFO - Chain [1] done processing


  Prophet: fit=0.404s predict=0.100s
Running benchmark for n=10000


...

01:05:40 - cmdstanpy - INFO - Chain [1] start processing
01:05:42 - cmdstanpy - INFO - Chain [1] done processing


  Prophet: fit=1.723s predict=0.339s
Running benchmark for n=50000


01:05:43 - cmdstanpy - INFO - Chain [1] start processing
01:05:46 - cmdstanpy - INFO - Chain [1] done processing


  Prophet: fit=3.844s predict=0.720s
Running benchmark for n=100000


OutOfBoundsDatetime: Cannot generate range with start=946684800000000000 and periods=100000

In [ ]:
# Cell 7: Results summary and plots
res_df = pd.DataFrame(results)
print(res_df)

# Plot fit times
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
if HAS_PROPHET:
    ax[0].plot(res_df['n'], res_df['prophet_fit_s'], 'o-', label='Prophet fit')
if HAS_SEER:
    ax[0].plot(res_df['n'], res_df['seer_fit_s'], 'o-', label='Seer fit')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_xlabel('Rows (log scale)')
ax[0].set_ylabel('Fit time (s, log scale)')
ax[0].set_title('Model Fit Time Scaling')
ax[0].legend()

# Plot predict times
if HAS_PROPHET:
    ax[1].plot(res_df['n'], res_df['prophet_predict_s'], 'o-', label='Prophet predict')
if HAS_SEER:
    ax[1].plot(res_df['n'], res_df['seer_predict_s'], 'o-', label='Seer predict')
ax[1].set_xscale('log')
ax[1].set_xlabel('Rows (log scale)')
ax[1].set_ylabel('Predict time (s)')
ax[1].set_title('Model Predict Time Scaling')
ax[1].legend()

plt.tight_layout()
plt.show()

# Save results to CSV for later analysis
res_df.to_csv('prophet_seer_benchmark_100k_results.csv', index=False)
print('Saved results to prophet_seer_benchmark_100k_results.csv')

## Notes and next steps

- This notebook focuses on raw runtime for fitting and predicting. For production comparisons consider memory usage, parallelism, and multi-series workloads.
- If you want monthly frequency tests, change `freq='D'` to `freq='MS'` in Cell 5; keep in mind `n=100k` months is unrealistic.
- Optionally add warm-start caching of model internals (if supported) to isolate predict-only costs.